<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Sizing of a multi-rotor drone

*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

The objective of this notebook is to select the best compromise of components (propeller, motor, ESC, battery) of a multi-rotor drone for given specifiations.

**Scipy** and **math** packages will be used for this notebook in order to illustrate the optimization algorithms of python.

In [1]:
import scipy
import scipy.optimize
from math import pi
from math import sqrt
import math
import timeit
import pandas as pd
from IPython.display import display, HTML
pd.options.display.float_format = '{:,.2f}'.format

## Sizing code

The set of equations of a sizing code can generate typical issues such : 
- Underconstrained set of equations: the lacking equations can come from additional scenarios, estimation models or additional sizing variable.   
- overconstrained equations often due to the selection of a component on multiple critera: the adding of over-sizing coefficients and constraints in the optimization problem can generally fix this issue   
- algebraic loops often due to selection criteria requiring informations generally available after the selection 

Concerning overconstraints components, we have here:
- Brushless motors with multiple torque and voltage constraints (hover and transient vertical displacement) 

Multiple algebraic loops appears in the sizing problem:
- The thrust depends of the total mass which depend of components required for generating this thrust

The final optimization problem depends thus of these parameters:
- $\beta_{pro}=pitch/diameter$ ratio to define the propeller
- $k_{os}$ over sizing coefficient on the load mass to estimate the final total mass
- $k_{mot}$ over sizing coeffcient on the motor torque to estimate the max torque with the hover flight conditions
- $k_{speed,mot}$ over sizing coeffcient on the motor speed to take into account voltage limits during hover or take-off flight
- $k_{ND}$ slow down propeller coef : ND = kNDmax / k_ND
- $k_{D}$ aspect ratio e_arm/D_out_arm (thickness/diameter) for the beam of the frame
- $k_{mb}$ ratio battery mass / payload mass
- $k_{vb}$ over sizing coefficient for the battery voltage

More details in the setting up of sizing code can be found in the  [following paper](https://www.researchgate.net/profile/Marc_Budinger/publication/277933677_Computer-aided_definition_of_sizing_procedures_and_optimization_problems_of_mechatronic_systems/links/55969de508ae793d137c7ea5/Computer-aided-definition-of-sizing-procedures-and-optimization-problems-of-mechatronic-systems.pdf):  

> Reysset, A., Budinger, M., & Maré, J. C. (2015). Computer-aided definition of sizing procedures and optimization problems of mechatronic systems. Concurrent Engineering, 23(4), 320-332.

The sizing code is defined here in a function which can give:
- an evaluation of the objective: here the total mass
- an evaluation of the constraints: 

Here is an non-exhaustive XDSM diagram of the multirotor sizing code:

![XDSM](pictures/xdsm_multirotor_mdo.png)

## Objectives and specifications

Main specifications :
- a load (video, control card) of mass $M_{load}$.  
- an autonomy $t_{hf}$ for the hover flight.
- an acceleration to take off $a_{to}$.


In [2]:
# Specifications

# Load
M_pay = 50. # [kg] load mass

# Acceleration during take off
a_to = 1. * 9.81 # [m/s²] acceleration

# Autonomy
t_hov_spec = 25. # [min] time of hover flight

# MTOW
MTOW = 310. # [kg] maximal mass

# Objectif
MAX_TIME = True # Objective

# Optimization bounds
# beta, k_os, k_mot, k_speed_mot, k_ND, k_D, k_mb, k_vb
opt_bounds = [(0.3,0.6), (1,10), (1,10), (1,10), (1,10), (0.05,.5), (.2,15), (1,5)]

## Architecture defintion and design assumptions

In [3]:
# Architecture of the multi-rotor drone (4,6, 8 arms, ...)
N_arm = 4 # [-] number of arm
N_pro_arm = 2 # [-] number of propeller per arm (1 or 2)
N_pro = N_pro_arm * N_arm # [-] Propellers number

In [4]:
from utils.model_serializer import ModelSerializer
from utils.model_standard import CoreModel, Model

In [5]:
# Loading models from .models directory
ms = ModelSerializer()
path = './models/'
file_name = 'motor_model'
motor_model = ms.load_model(path + file_name)
file_name = 'propeller_model'
propeller_model = ms.load_model(path + file_name)
file_name = 'battery_esc_model'
battery_esc_model = ms.load_model(path + file_name)
file_name = 'frame_model'
frame_model = ms.load_model(path + file_name)

In [6]:
# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else
class SizingCode(Model):
    
    def __init__(self, name, **kwargs):
        super(SizingCode, self).__init__(name, **kwargs)
        self.optimization_data_frame = pd.DataFrame()
       
    def initialization(self):
        # Input variables
        self.add_input('k_os', value=1.0, unit='-', comment='Over sizing coefficient on the load mass')
        self.add_input('k_vb', value=1.0, unit='-', comment='Over sizing coefficient for the battery voltage')
        self.add_input('t_hov_spec', value=10.0, unit='-', comment='Specified hovering time')
        
        # Output variables
        self.add_output('M_total', unit='kg', comment='First total mass estimation of the multirotor')
        self.add_output('M_total_real', unit='kg', comment='Effective total mass of the multirotor')

        # Adding submodels
        self.add_submodel(motor_model)
        self.add_submodel(propeller_model)
        self.add_submodel(battery_esc_model)
        self.add_submodel(frame_model)
        
    def print_optimization(self):
        # Functions that displays information related to the optimization
        p = self.parameters
        col_names = ['Type', 'Name', 'Min', 'Value', 'Max', 'Unit', 'Comment']
        
        df = self.optimization_data_frame
        
        # Design variables
        df = df.append([{'Type': 'DV', 'Name': 'beta_pro', 'Min': opt_bounds[0][0], 'Value': p['beta_pro'], 'Max': opt_bounds[0][1], 'Unit': '[-]', 'Comment': 'pitch/diameter ratio of the propeller'}])[col_names]
        df = df.append([{'Type': 'DV', 'Name': 'k_os', 'Min': opt_bounds[1][0], 'Value': p['k_os'], 'Max': opt_bounds[1][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the load mass'}])
        df = df.append([{'Type': 'DV', 'Name': 'k_mot', 'Min': opt_bounds[2][0], 'Value': p['k_mot'], 'Max': opt_bounds[2][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the motor torque'}])
        df = df.append([{'Type': 'DV', 'Name': 'k_speed_mot', 'Min': opt_bounds[3][0], 'Value': p['k_speed_mot'], 'Max': opt_bounds[3][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the motor speed'}])
        df = df.append([{'Type': 'DV', 'Name': 'k_ND', 'Min': opt_bounds[4][0], 'Value': p['k_ND'], 'Max': opt_bounds[4][1], 'Unit': '[-]', 'Comment': 'slow down propeller coef : ND = kNDmax / k_ND'}])
        df = df.append([{'Type': 'DV', 'Name': 'k_D', 'Min': opt_bounds[5][0], 'Value': p['k_D'], 'Max': opt_bounds[5][1], 'Unit': '[-]', 'Comment': 'aspect ratio e_arm/D_out_arm (thickness/diameter) for the beam of the frame'}])
        df = df.append([{'Type': 'DV', 'Name': 'k_mb', 'Min': opt_bounds[6][0], 'Value': p['k_mb'], 'Max': opt_bounds[6][1], 'Unit': '[-]', 'Comment': 'ratio battery / payload mass'}])
        df = df.append([{'Type': 'DV', 'Name': 'k_vb', 'Min': opt_bounds[7][0], 'Value': p['k_vb'], 'Max': opt_bounds[7][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient for the battery voltage'}])
        
        # Constraints
        df = df.append([{'Type': 'CONSTR', 'Name': 'Constr 1', 'Min': 0.0, 'Value': p['M_total']-p['M_total_real'], 'Max': '-', 'Unit': '[kg]', 'Comment': 'Mass sizing loop'}])
        df = df.append([{'Type': 'CONSTR', 'Name': 'Constr 2', 'Min': 0.0, 'Value': p['U_bat']-p['U_mot_to'], 'Max': '-', 'Unit': '[V]', 'Comment': 'Battery/Motor voltage'}])
        df = df.append([{'Type': 'CONSTR', 'Name': 'Constr 3', 'Min': 0.0, 'Value': p['T_max_mot']-p['T_pro_to'], 'Max': '-', 'Unit': '[N.m]', 'Comment': 'Motor max torque'}])
        df = df.append([{'Type': 'CONSTR', 'Name': 'Constr 4', 'Min': 0.0, 'Value': p['U_bat']-p['V_esc'], 'Max': '-', 'Unit': '[V]', 'Comment': 'Battery/ESC voltage'}])
        
        # Objective
        df = df.append([{'Type': 'OBJ', 'Name': 't_hov', 'Min': '-', 'Value': p['t_hov'], 'Max': '-', 'Unit': '[min]', 'Comment': 'Hovering time (autonomy)'}])
        display(df)
        
    def computation_script(self, param, arg):
        p = self.parameters
        
        # Design variables
        p['beta_pro'] = param[0] # pitch/diameter ratio of the propeller
        p['k_os'] = param[1] # over sizing coefficient on the load mass 
        p['k_mot'] = param[2] # over sizing coefficient on the motor torque
        p['k_speed_mot'] = param[3] # over sizing coefficient on the motor speed
        p['k_ND'] = param[4] # slow down propeller coef : ND = kNDmax / k_ND
        p['k_D'] = param[5] # aspect ratio e_arm/D_out_arm (thickness/diameter) for the beam of the frame
        p['k_mb'] = param[6] # ratio battery / payload mass 
        p['k_vb'] = param[7] # over sizing coefficient for the battery voltage
        
        # Input parameters 
        p['M_pay'] = M_pay
        p['a_to'] = a_to
        p['t_hov_spec'] = t_hov_spec
        p['MTOW'] = MTOW
        p['N_arm'] = N_arm
        p['N_pro_arm'] = N_pro_arm
        p['N_pro'] = N_pro
 
        # Hover & Take-Off thrust 
        p['M_total'] = p['k_os'] * p['M_pay'] # [kg] Estimation of the total mass (or equivalent weight of dynamic scenario)

        p['F_pro_hov'] = p['M_total'] * (9.81) / p['N_pro'] # [N] Thrust per propeller for hover
        p['F_pro_to'] = p['M_total'] * (9.81+ p['a_to']) / p['N_pro'] # [N] Thrust per propeller for take-off

        # Propeller sizing
        self.run_submodel('propeller')

        # Battery voltage estimation with propeller power
        p['U_bat_est'] = p['k_vb']*1.84*(p['P_pro_to'])**(0.36) # [V] battery voltage estimation

        # Motor sizing
        self.run_submodel('motor')

        # Battery and ESC sizing  
        self.run_submodel('battery_esc')

        # Frame sizing
        self.run_submodel('frame')

        # Objective and Constraints sum up
        p['M_total_real'] = (p['M_esc']+p['M_pro']+p['M_mot'])*p['N_pro']+p['M_pay']+p['M_bat']+p['M_frame']

        if MAX_TIME == True:
            constraints = [p['M_total']-p['M_total_real'], p['U_bat']-p['U_mot_to'], p['T_max_mot']-p['T_pro_to'], p['U_bat']-p['V_esc'], p['MTOW']-p['M_total_real']]
        else:
            constraints = [p['M_total']-p['M_total_real'], p['U_bat']-p['U_mot_to'], p['T_max_mot']-p['T_pro_to'], p['U_bat']-p['V_esc'], p['t_hov']-p['t_hov_spec']]

        # Objective and contraints
        if arg == 'Obj':
            if MAX_TIME == True:
                return 1./p['t_hov'] # for time maximisation
            else:
                return p['M_total_real'] # for mass optimisation
            
         # Objective and contraints
        if arg == 'ObjP':
            P = 0. # Penalisation nulle
            for C in constraints: 
                if (C < 0.): 
                    P = P-1e9*C
            if MAX_TIME==True:
                return 1./p['t_hov'] + P # for time maximisation
            else:
                return p['M_total_real'] + P # for mass optimisation       
        else:
            return constraints


## Optimization problem


We will now use the [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLSQP algorithm without explicit expression of the gradient (Jacobian). A course on Multidisplinary Gradient optimization algorithms and gradient optimization algorithm is given [here](http://mdolab.engin.umich.edu/sites/default/files/Martins-MDO-course-notes.pdf):
> Joaquim R. R. A. Martins (2012). A Short Course on Multidisciplinary Design Optimization. University of Michigan


The first step is to give an initial value of optimisation variables:

In [7]:
# Optimisation variables
beta_pro = .33 # pitch/diameter ratio of the propeller
k_os = 3.2 # over sizing coefficient on the load mass 
k_mot = 1. # over sizing coefficient on the motor torque
k_speed_mot = 1.2 # adaption winding coef on the motor speed
k_ND = 1. # reduction of product speed.diameter on the propeller
k_D = .01 # aspect ratio e/c (thickness/side) for the beam of the frame
k_mb = 1. # ratio battery/load mass
k_vb = 1. # oversizing coefficient for voltage evaluation

# Vector of parameters
parameters = scipy.array((beta_pro, k_os, k_mot, k_speed_mot, k_ND, k_D, k_mb, k_vb))

We can print of the characterisitcs of the problem before optimization with the initial vector of optimization variables:

In [8]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")

sizing_code = SizingCode('multirotor')
sizing_code.initialization()

objectif = sizing_code.computation_script(parameters, 'Obj')
sizing_code.print_optimization()
sizing_code.print_variables()
print("-----------------------------------------------")

-----------------------------------------------
Initial characteristics before optimization :


,Type,Name,Min,Value,Max,Unit,Comment
0,DV,beta_pro,0.30,0.33,0.60,[-],pitch/diameter ratio of the propeller
0,DV,k_os,1.00,3.20,10.00,[-],over sizing coefficient on the load mass
0,DV,k_mot,1.00,1.00,10.00,[-],over sizing coefficient on the motor torque
0,DV,k_speed_mot,1.00,1.20,10.00,[-],over sizing coefficient on the motor speed
0,DV,k_ND,1.00,1.00,10.00,[-],slow down propeller coef : ND = kNDmax / k_ND
0,DV,k_D,0.05,0.01,0.50,[-],aspect ratio e_arm/D_out_arm (thickness/diamet...
0,DV,k_mb,0.20,1.00,15.00,[-],ratio battery / payload mass
0,DV,k_vb,1.00,1.00,5.00,[-],over sizing coefficient for the battery voltage
0,CONSTR,Constr 1,0.00,16.59,-,[kg],Mass sizing loop
0,CONSTR,Constr 2,0.00,-2.51,-,[V],Battery/Motor voltage


interactive(children=(Dropdown(description='Component', options=('multirotor', 'battery_esc', 'propeller', 'fr…

-----------------------------------------------


Then we can solve the problem and print of the optimized solution:

In [9]:
# Optimization with SLSQP algorithm
contrainte = lambda x: sizing_code.computation_script(x, 'Const')
objectif = lambda x: sizing_code.computation_script(x, 'Obj')
objectifP = lambda x: sizing_code.computation_script(x, 'ObjP')

SLSQP = False # Optimization algorithm choice

if SLSQP == True:
    # SLSQP omptimisation
    result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=opt_bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)
else:
    # Differential evolution omptimisation
    result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=opt_bounds,
                                   tol=1e-12)

# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

if SLSQP == True:
    sizing_code.computation_script(result,'Obj')
    sizing_code.print_optimization()
    sizing_code.print_variables()
else:
    sizing_code.print_optimization()
    sizing_code.print_variables()
print("-----------------------------------------------")



-----------------------------------------------
Final characteristics after optimization :


,Type,Name,Min,Value,Max,Unit,Comment
0,DV,beta_pro,0.30,0.30,0.60,[-],pitch/diameter ratio of the propeller
0,DV,k_os,1.00,6.20,10.00,[-],over sizing coefficient on the load mass
0,DV,k_mot,1.00,1.65,10.00,[-],over sizing coefficient on the motor torque
0,DV,k_speed_mot,1.00,1.09,10.00,[-],over sizing coefficient on the motor speed
0,DV,k_ND,1.00,1.00,10.00,[-],slow down propeller coef : ND = kNDmax / k_ND
0,DV,k_D,0.05,0.05,0.50,[-],aspect ratio e_arm/D_out_arm (thickness/diamet...
0,DV,k_mb,0.20,2.12,15.00,[-],ratio battery / payload mass
0,DV,k_vb,1.00,1.05,5.00,[-],over sizing coefficient for the battery voltage
0,CONSTR,Constr 1,0.00,0.00,-,[kg],Mass sizing loop
0,CONSTR,Constr 2,0.00,0.00,-,[V],Battery/Motor voltage


interactive(children=(Dropdown(description='Component', options=('multirotor', 'battery_esc', 'propeller', 'fr…

-----------------------------------------------
